# <span style="font-size:1.5em;"> Forecasting Air Quality in New York City
Author: Angela Kim

---

# <span style="font-size:1.2em;"> Contents
<l></l>

<span style="font-size:1.2em;">

- <a href="#Overview">Overview</a>
    
- <a href="#Imports">Imports</a>
    
    - <a href="#Ozone">Ozone</a>
    
    - <a href="#Carbon Monoxide">Carbon Monoxide</a>
    
    - <a href="#Nitrogen Dioxide">Nitrogen Dioxide</a>

    - <a href="#Sulfur Dioxide">Sulfur Dioxide</a>
    
- <a href="#Creating Trim Pollutant Datasets">Creating Trim Pollutant Datasets</a>

- <a href="#Creating Time Series Datasets">Creating Time Series Datasets</a>
    
- <a href="#Export">Export</a>

- <a href="#Sources">Sources</a>

# <span style="font-size:1.2em;"> <a id="Overview">Overview</a>

# <span style="font-size:1.2em;"> <a id="Imports">Imports</a>

In [2]:
# Import libraries
import pandas as pd
import numpy as np
import itertools
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error as mse

from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_process import ArmaProcess
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

In [3]:
O3 = pd.read_csv('data/nycO3.csv')
CO = pd.read_csv('data/nycCO.csv')
NO2 = pd.read_csv('data/nycNO2.csv')
SO2 = pd.read_csv('data/nycSO2.csv')

## <span style="font-size:1.2em;"> <a id="Ozone">Ozone</a>

In [2]:
# Import Ozone datasets
O3_2000 = pd.read_csv('data/dailyO3/daily_44201_2000.csv')
O3_2001 = pd.read_csv('data/dailyO3/daily_44201_2001.csv')
O3_2002 = pd.read_csv('data/dailyO3/daily_44201_2002.csv')
O3_2003 = pd.read_csv('data/dailyO3/daily_44201_2003.csv')
O3_2004 = pd.read_csv('data/dailyO3/daily_44201_2004.csv')
O3_2005 = pd.read_csv('data/dailyO3/daily_44201_2005.csv')
O3_2006 = pd.read_csv('data/dailyO3/daily_44201_2006.csv')
O3_2007 = pd.read_csv('data/dailyO3/daily_44201_2007.csv')
O3_2008 = pd.read_csv('data/dailyO3/daily_44201_2008.csv')
O3_2009 = pd.read_csv('data/dailyO3/daily_44201_2009.csv')
O3_2010 = pd.read_csv('data/dailyO3/daily_44201_2010.csv')
O3_2011 = pd.read_csv('data/dailyO3/daily_44201_2011.csv')
O3_2012 = pd.read_csv('data/dailyO3/daily_44201_2012.csv')
O3_2013 = pd.read_csv('data/dailyO3/daily_44201_2013.csv')
O3_2014 = pd.read_csv('data/dailyO3/daily_44201_2014.csv')
O3_2015 = pd.read_csv('data/dailyO3/daily_44201_2015.csv')
O3_2016 = pd.read_csv('data/dailyO3/daily_44201_2016.csv')
O3_2017 = pd.read_csv('data/dailyO3/daily_44201_2017.csv')
O3_2018 = pd.read_csv('data/dailyO3/daily_44201_2018.csv')
O3_2019 = pd.read_csv('data/dailyO3/daily_44201_2019.csv')
O3_2020 = pd.read_csv('data/dailyO3/daily_44201_2020.csv')
O3_2021 = pd.read_csv('data/dailyO3/daily_44201_2021.csv')


# Concatenate datasets
O3_all = [O3_2000, O3_2001, O3_2002, O3_2003, O3_2004, O3_2005, O3_2006, O3_2007, O3_2008, O3_2009, O3_2010, 
          O3_2011, O3_2012, O3_2013, O3_2014, O3_2015, O3_2016, O3_2017, O3_2018, O3_2019, O3_2020, O3_2021]

O3 = pd.concat(O3_all, ignore_index=True)

## <span style="font-size:1.2em;"> <a id="Carbon Monoxide">Carbon Monoxide</a>

In [3]:
# Import Carbon Monoxide datasets
CO_2000 = pd.read_csv('data/dailyCO/daily_42101_2000.csv')
CO_2001 = pd.read_csv('data/dailyCO/daily_42101_2001.csv')
CO_2002 = pd.read_csv('data/dailyCO/daily_42101_2002.csv')
CO_2003 = pd.read_csv('data/dailyCO/daily_42101_2003.csv')
CO_2004 = pd.read_csv('data/dailyCO/daily_42101_2004.csv')
CO_2005 = pd.read_csv('data/dailyCO/daily_42101_2005.csv')
CO_2006 = pd.read_csv('data/dailyCO/daily_42101_2006.csv')
CO_2007 = pd.read_csv('data/dailyCO/daily_42101_2007.csv')
CO_2008 = pd.read_csv('data/dailyCO/daily_42101_2008.csv')
CO_2009 = pd.read_csv('data/dailyCO/daily_42101_2009.csv')
CO_2010 = pd.read_csv('data/dailyCO/daily_42101_2010.csv')
CO_2011 = pd.read_csv('data/dailyCO/daily_42101_2011.csv')
CO_2012 = pd.read_csv('data/dailyCO/daily_42101_2012.csv')
CO_2013 = pd.read_csv('data/dailyCO/daily_42101_2013.csv')
CO_2014 = pd.read_csv('data/dailyCO/daily_42101_2014.csv')
CO_2015 = pd.read_csv('data/dailyCO/daily_42101_2015.csv')
CO_2016 = pd.read_csv('data/dailyCO/daily_42101_2016.csv')
CO_2017 = pd.read_csv('data/dailyCO/daily_42101_2017.csv')
CO_2018 = pd.read_csv('data/dailyCO/daily_42101_2018.csv')
CO_2019 = pd.read_csv('data/dailyCO/daily_42101_2019.csv')
CO_2020 = pd.read_csv('data/dailyCO/daily_42101_2020.csv')
CO_2021 = pd.read_csv('data/dailyCO/daily_42101_2021.csv')


# Concatenate datasets
CO_all = [CO_2000, CO_2001, CO_2002, CO_2003, CO_2004, CO_2005, CO_2006, CO_2007, CO_2008, CO_2009, CO_2010,
          CO_2011, CO_2012, CO_2013, CO_2014, CO_2015, CO_2016, CO_2017, CO_2018, CO_2019, CO_2020, CO_2021]

CO = pd.concat(CO_all, ignore_index=True)

## <span style="font-size:1.2em;"> <a id="Nitrogen Dioxide">Nitrogen Dioxide</a>

In [4]:
# Import Nitrogen Dioxide datasets
NO2_2000 = pd.read_csv('data/dailyNO2/daily_42602_2000.csv')
NO2_2001 = pd.read_csv('data/dailyNO2/daily_42602_2001.csv')
NO2_2002 = pd.read_csv('data/dailyNO2/daily_42602_2002.csv')
NO2_2003 = pd.read_csv('data/dailyNO2/daily_42602_2003.csv')
NO2_2004 = pd.read_csv('data/dailyNO2/daily_42602_2004.csv')
NO2_2005 = pd.read_csv('data/dailyNO2/daily_42602_2005.csv')
NO2_2006 = pd.read_csv('data/dailyNO2/daily_42602_2006.csv')
NO2_2007 = pd.read_csv('data/dailyNO2/daily_42602_2007.csv')
NO2_2008 = pd.read_csv('data/dailyNO2/daily_42602_2008.csv')
NO2_2009 = pd.read_csv('data/dailyNO2/daily_42602_2009.csv')
NO2_2010 = pd.read_csv('data/dailyNO2/daily_42602_2010.csv')
NO2_2011 = pd.read_csv('data/dailyNO2/daily_42602_2011.csv')
NO2_2012 = pd.read_csv('data/dailyNO2/daily_42602_2012.csv')
NO2_2013 = pd.read_csv('data/dailyNO2/daily_42602_2013.csv')
NO2_2014 = pd.read_csv('data/dailyNO2/daily_42602_2014.csv')
NO2_2015 = pd.read_csv('data/dailyNO2/daily_42602_2015.csv')
NO2_2016 = pd.read_csv('data/dailyNO2/daily_42602_2016.csv')
NO2_2017 = pd.read_csv('data/dailyNO2/daily_42602_2017.csv')
NO2_2018 = pd.read_csv('data/dailyNO2/daily_42602_2018.csv')
NO2_2019 = pd.read_csv('data/dailyNO2/daily_42602_2019.csv')
NO2_2020 = pd.read_csv('data/dailyNO2/daily_42602_2020.csv')
NO2_2021 = pd.read_csv('data/dailyNO2/daily_42602_2021.csv')


# Concatenate datasets
NO2_all = [NO2_2000, NO2_2001, NO2_2002, NO2_2003, NO2_2004, NO2_2005, NO2_2006, NO2_2007, NO2_2008, NO2_2009, 
           NO2_2010, NO2_2011, NO2_2012, NO2_2013, NO2_2014, NO2_2015, NO2_2016, NO2_2017, NO2_2018, NO2_2019, 
           NO2_2020, NO2_2021]

NO2 = pd.concat(NO2_all, ignore_index=True)

## <span style="font-size:1.2em;"> <a id="Sulfur Dioxide">Sulfur Dioxide</a>

In [5]:
# Import Sulfur Dioxide datasets
SO2_2000 = pd.read_csv('data/dailySO2/daily_42401_2000.csv')
SO2_2001 = pd.read_csv('data/dailySO2/daily_42401_2001.csv')
SO2_2002 = pd.read_csv('data/dailySO2/daily_42401_2002.csv')
SO2_2003 = pd.read_csv('data/dailySO2/daily_42401_2003.csv')
SO2_2004 = pd.read_csv('data/dailySO2/daily_42401_2004.csv')
SO2_2005 = pd.read_csv('data/dailySO2/daily_42401_2005.csv')
SO2_2006 = pd.read_csv('data/dailySO2/daily_42401_2006.csv')
SO2_2007 = pd.read_csv('data/dailySO2/daily_42401_2007.csv')
SO2_2008 = pd.read_csv('data/dailySO2/daily_42401_2008.csv')
SO2_2009 = pd.read_csv('data/dailySO2/daily_42401_2009.csv')
SO2_2010 = pd.read_csv('data/dailySO2/daily_42401_2010.csv')
SO2_2011 = pd.read_csv('data/dailySO2/daily_42401_2011.csv')
SO2_2012 = pd.read_csv('data/dailySO2/daily_42401_2012.csv')
SO2_2013 = pd.read_csv('data/dailySO2/daily_42401_2013.csv')
SO2_2014 = pd.read_csv('data/dailySO2/daily_42401_2014.csv')
SO2_2015 = pd.read_csv('data/dailySO2/daily_42401_2015.csv')
SO2_2016 = pd.read_csv('data/dailySO2/daily_42401_2016.csv')
SO2_2017 = pd.read_csv('data/dailySO2/daily_42401_2017.csv')
SO2_2018 = pd.read_csv('data/dailySO2/daily_42401_2018.csv')
SO2_2019 = pd.read_csv('data/dailySO2/daily_42401_2019.csv')
SO2_2020 = pd.read_csv('data/dailySO2/daily_42401_2020.csv')
SO2_2021 = pd.read_csv('data/dailySO2/daily_42401_2021.csv')


# Concatenate datasets
SO2_all = [SO2_2000, SO2_2001, SO2_2002, SO2_2003, SO2_2004, SO2_2005, SO2_2006, SO2_2007, SO2_2008, SO2_2009, 
           SO2_2010, SO2_2011, SO2_2012, SO2_2013, SO2_2014, SO2_2015, SO2_2016, SO2_2017, SO2_2018, SO2_2019, 
           SO2_2020, SO2_2021]

SO2 = pd.concat(SO2_all, ignore_index=True)

# <span style="font-size:1.2em;"> <a id="Creating Trim Pollutant Datasets">Creating Trim Pollutant Datasets</a>

In [6]:
# Function to trim all four pollutant datasets

def trim_dataset(df, pollutant):
    """
    Trims down pollution datasets.
    
    Parameters:
    ----------
    df: DataFrame
    pollutant: str, name of pollutant
    
    Returns:
    ----------
    DataFrame
    """
    
    # Drop columns that are not US states or DC
    df.drop(df[(df['State Name'] == 'Country Of Mexico') | 
               (df['State Name'] == 'Virgin Islands') | 
               (df['State Name'] == 'Canada') | 
               (df['State Name'] == 'Puerto Rico')].index, inplace=True)
    
    # Drop pollutant standards that do not produce AQI values
    if pollutant == 'CO':
        df.drop(df[df['Pollutant Standard'] == ('CO 1-hour 1971')].index, inplace=True)
    elif pollutant == 'SO2':
        df.drop(df[df['Pollutant Standard'] == ('SO2 3-hour 1971')].index, inplace=True)
    else:
        pass
    
    # Drop columns that are redundant or unnecessary
    df.drop(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Latitude', 'Longitude', 
             'Datum', 'Parameter Name', 'Sample Duration', 'Pollutant Standard', 'Units of Measure', 
             'Event Type', 'Observation Count', 'Observation Percent', 'Method Code', 'Method Name', 
             'Local Site Name', 'Address', 'CBSA Name', 'Date of Last Change'], axis=1, inplace=True)
    
    # Reorder columns for neatness
    reordered = ['Date Local', 'State Name', 'County Name', 'City Name', 
                 'Arithmetic Mean', '1st Max Value', '1st Max Hour', 'AQI']
    
    df = df.reindex(columns=reordered)
    
    # Rename columns for neatness
    df = df.rename(columns={'Date Local': 'Date', 
                            'State Name': 'State', 
                            'County Name': 'County', 
                            'City Name': 'City', 
                            'Arithmetic Mean': 'Mean', 
                            'AQI': '{} AQI'.format(pollutant)})
    
    return df

In [7]:
# Applying the function to datasets
O3trim = trim_dataset(O3, 'O3')
COtrim = trim_dataset(CO, 'CO')
NO2trim = trim_dataset(NO2, 'NO2')
SO2trim = trim_dataset(SO2, 'SO2')

# <span style="font-size:1.2em;"> <a id="Creating Time Series Datasets">Creating Time Series Datasets</a>

In [8]:
# Function to make time series datasets

def ts_dataset(df, pollutant):
    """
    Creates time series datasets from trimmed pollutant datasets.
    
    Parameters:
    ----------
    df: DataFrame
    pollutant: str, name of pollutant
    
    Returns:
    ----------
    DataFrame
    """
    
    # Drop mean, max value, and max hour columns
    df.drop(['Mean', '1st Max Value', '1st Max Hour'], axis=1, inplace=True)
    
    # Set date as index
    df.set_index(['Date'], inplace=True)
    
    return df

In [9]:
# Applying the function to create time series datasets
O3ts = ts_dataset(O3trim, 'O3')
COts = ts_dataset(COtrim, 'CO')
NO2ts = ts_dataset(NO2trim, 'NO2')
SO2ts = ts_dataset(SO2trim, 'SO2')

In [10]:
# Function to make NYC time series datasets

def nyc_ts_dataset(df, pollutant):
    """
    Creates NYC time series datasets from time series datasets.
    
    Parameters:
    ----------
    df: DataFrame
    pollutant: str, name of pollutant
    
    Returns:
    ----------
    DataFrame
    """
    
    # Keep rows for NYC then drop location columns
    df = df[df['City']=='New York']
    df.drop(['State', 'County', 'City'], axis=1, inplace=True)
    
    # Groupby date and take max AQI values if there are duplicates of the same date
    df = df.groupby('Date').max()
    
    return df

In [11]:
# Applying the function to create NYC time series datasets
nycO3 = nyc_ts_dataset(O3ts, 'O3')
nycCO = nyc_ts_dataset(COts, 'CO')
nycNO2 = nyc_ts_dataset(NO2ts, 'NO2')
nycSO2 = nyc_ts_dataset(SO2ts, 'SO2')

# <span style="font-size:1.2em;"> <a id="Export">Export</a>

In [12]:
# Export individual pollutant datasets
# O3.to_csv('O3.csv', index=False)
# CO.to_csv('CO.csv', index=False)
# NO2.to_csv('NO2.csv', index=False)
# SO2.to_csv('SO2.csv', index=False)

# Export trim pollutant datasets
# O3.to_csv('O3trim.csv', index=False)
# CO.to_csv('COtrim.csv', index=False)
# NO2.to_csv('NO2trim.csv', index=False)
# SO2.to_csv('SO2trim.csv', index=False)

# Export time series datasets
# O3ts.to_csv('O3ts.csv')
# COts.to_csv('COts.csv')
# NO2ts.to_csv('NO2ts.csv')
# SO2ts.to_csv('SO2ts.csv')

# Export NYC time series datasets
# nycO3.to_csv('nycO3.csv')
# nycCO.to_csv('nycCO.csv')
# nycNO2.to_csv('nycNO2.csv')
# nycSO2.to_csv('nycSO2.csv')

# <span style="font-size:1.2em;"> <a id="Sources">Sources</a>

- [EPA AirData Daily Summary Data](https://aqs.epa.gov/aqsweb/airdata/download_files.html#Daily)